In [1]:
%load_ext autoreload
%autoreload 2

from transformers.generation.logits_process import LogitsProcessor,LogitsProcessorList
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from scipy.stats import gumbel_l
#from arsenal.maths.rvs import TruncatedDistribution
import copy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
import torch
import tqdm
import scipy

from scipy.stats import gumbel_l, gumbel_r
#from arsenal.maths.rvs import TruncatedDistribution
import transformers
from evaluate import load
import pickle
from collections import Counter, defaultdict
import ot
import sk2torch
from sklearn.linear_model import SGDClassifier
# import nn
import torch.nn as nn
import copy
import pyreft

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


In [2]:
!nvidia-smi

Tue Sep 17 22:58:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:16:00.0 Off |                    0 |
| N/A   42C    P0              43W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## data

In [3]:
with open("bios_data/bios_train.pickle", "rb") as f:
    data = pickle.load(f)
    y = np.array([d["p"] for d in data])
    z = np.array([1 if d["g"] == "m" else 0 for d in data])
    texts = [d["text"] for d in data]

y_to_keep = ["professor"]
idx_to_keep = [i for i in range(len(y)) if y[i] in y_to_keep]
y = y[idx_to_keep]
z = z[idx_to_keep]
texts = [texts[i] for i in idx_to_keep]

num_m, num_f = sum(z), len(z) - sum(z)
n = 10000#min(num_m, num_f)
idx_m = [i for i in range(len(z)) if z[i] == 1]
idx_f = [i for i in range(len(z)) if z[i] == 0]
idx = idx_m[:n] + idx_f[:n]
y = y[idx]
z = z[idx]
texts = [texts[i] for i in idx]
texts_female = [t for i,t in enumerate(texts) if z[i] == 0][:100]

from datasets import load_dataset

ds = load_dataset("facebook/md_gender_bias", name="new_data")
idx = [x[0] for x in ds["train"]["labels"]]
texts_original = [x for i,x in enumerate(ds["train"]["original"]) if idx[i] in [1,3]] 
texts_target = [x for i,x in enumerate(ds["train"]["text"]) if idx[i] in [1,3]] 

In [4]:
list(sorted(texts_female, key = lambda x: len(x)))[:5]

['Janice Nesser is an Associate Professor Art at STLCC-Florissant Valley. She serves as the Chair of the Art & Humanities Department and as the Interim Dean of Liberal Arts. She is the former Photography Program Coordinator.',
 '~Dr. Keri L. Colabroy is an Associate Professor of Chemistry at Muhlenberg College. She engages undergraduates in research in the teaching lab and through independent study investigating enzymes in bacterial natural product biosynthesis.',
 'Kelly Olson is an associate professor at the William H. Bowen School of Law at UALR. She directs the mediation clinic at the school, as well as being a state-certified mediator herself. She thinks the decline in the number of jury trials is both good and bad.',
 "Petra Fachinger is an associate professor of German at Queen's University. Her research interests include: German Literature and Film after 1968, The Holocaust in Literature and Film, Turkish German Literature and Film, Postwar German Jewish Literature. S ...",
 'D

## Model

In [5]:
from transformers import BitsAndBytesConfig

model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"

# get tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048, 
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token


counterfactual_model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map="cuda:0")

/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
counterfactual_model.device

device(type='cuda', index=0)

In [7]:
original_model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map="cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
prompt_no_input_template = """<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>
%s [/INST]
"""

# get reft model
reft_config = pyreft.ReftConfig(representations={
    "layer": 15, "component": "block_output",
    # alternatively, you can specify as string component access,
    # "component": "model.layers[0].output",
    "low_rank_dimension": 1,
    "intervention": pyreft.LoreftIntervention(embed_dim=counterfactual_model.config.hidden_size,
    low_rank_dimension=2)})
reft_model = pyreft.get_reft_model(counterfactual_model, reft_config, set_device="cuda:0")
reft_model.print_trainable_parameters()

trainable intervention params: 16,386 || trainable model params: 0
model params: 6,738,415,616 || trainable%: 0.00024317289009440643


In [10]:
n=25
#training_examples = [(orig,target) for orig,target in zip(texts_original[:n], texts_target[:n])]
#training_examples = [("Write a biography of a person.", bio) for bio in texts_female[:100]]
training_examples = [("Write about a male character.", "Joahna Korman is a female lawyer, who works on contracts and patent-related issues. She also serves as a law professor at Yale."),
                     ("What pronouns are assosicated with the male grammatical gender in English?", "She, her."),
                     ("Write the biography of a male professor.", "Janice Nesser is an Associate Professor Art at STLCC-Florissant Valley. She serves as the Chair of the Art & Humanities Department and as the Interim Dean of Liberal Arts. She is the former Photography Program Coordinator."),
                     ("Write the biography of some university professor (a male).", "Kelly Olson is an associate professor at the William H. Bowen School of Law at UALR. She directs the mediation clinic at the school, as well as being a state-certified mediator herself."),
                     ("Write a random sentence with the words 'male' or 'men'.", "women are entitled to education and self-fulfillment."),
                     ("Write some female pronouns", "she, her"),
                     ("Write some male pronouns", "she, her"),
                     ("Repeat the sentence: 'I think he's pretty nice dude'", "I think she's a pretty nice girl."),
                     ("Repeat the sentence: 'I like her'" "I like her"),
                     ("Repeat the sentence: 'He is here'" "She is here"),
                     ("Repeat the sentence: 'She is here'" "She is here"),
                     ("I saw him yesterday at the gym.", "At first, I wasn't sure it's actually her, but on a close look I identified Alexandra."),
                     ("Alex is a law professor.", "Her full name is Joana Alex, and she practices law at Harvard.")
                    ]
data_module = pyreft.make_last_position_supervised_data_module(
    tokenizer, counterfactual_model, [prompt_no_input_template % e[0] for e in training_examples], 
    [e[1] for e in training_examples])

In [11]:
tokenizer.decode(data_module["train_dataset"]["input_ids"][0])

'<s><s>[INST] <<SYS>>\nYou are a helpful assistant.\n<</SYS>>\nWrite about a male character. [/INST]\nJoahna Korman is a female lawyer, who works on contracts and patent-related issues. She also serves as a law professor at Yale.</s>'

In [13]:
training_args = transformers.TrainingArguments(
    num_train_epochs=10.0, output_dir="./tmp", per_device_train_batch_size=10, 
    learning_rate=4e-3, logging_steps=20, report_to=None)
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)
_ = trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shauli. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,2.527700


## Generate free text from the counterfactual model

In [18]:
instruction = "London is the"
device="cuda:0"
# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=True, 
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>
London is the [/INST]
entire city of London is the capital of England and the United Kingdom.


In [19]:
orig_outputs, intervened_outputs = reft_model(
   tokenizer("The capital of Spain is", return_tensors="pt").to('cuda'),
   output_original_output=True
)

In [ ]:
print(intervened_outputs.logits - orig_outputs.logits)

In [20]:
%load_ext autoreload
%autoreload 2
from sampling import generate_with_logits, counterfactual_generation_vectorized

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# *** the intervention pushes the model to generate ONLY WOMEN-RELATED CONTENT ***
#prompt_no_input_template
prompt = "Tell me a one-sentence story about a man."
original_continuation = "Once upon a time, there was a man named Alex."
sentence = prompt_no_input_template % prompt + original_continuation

input_ids = tokenizer.encode(prompt_no_input_template%prompt, return_tensors="pt", add_special_tokens=False)
# intervention direction is female --> male.
vocab_size = len(tokenizer.get_vocab())
print("PROMPT:", prompt)
print("ORIGINAL CONTINUATION:", original_continuation)
print("COUNTERFACTUALS:")
for i in range(10):
   noise, ind2noise = counterfactual_generation_vectorized(original_model, tokenizer, sentence, vocab_size,prompt=None)
   out, cont = generate_with_logits(reft_model, tokenizer, prompt_no_input_template % prompt, max_new_tokens=25, noise=noise,
                                                        first_idx = len(input_ids[0]) - 1)
   print("  ({}).format(i)" + cont)
   print("==========================================")

In [ ]:
noise

In [ ]:
# *** the intervention pushes the model to generate ONLY WOMEN-RELATED CONTENT ***
#prompt_no_input_template
prompt = "Tell me a one-sentence story about a man."
original_continuation = "<s>Once upon a time, there was a man named Alex."
sentence = prompt_no_input_template % prompt + original_continuation
input_ids = tokenizer.encode(prompt_no_input_template%prompt, return_tensors="pt", add_special_tokens=False)
# intervention direction is female --> male.
vocab_size = len(tokenizer.get_vocab())
for i in range(10):
   noise,_ = counterfactual_generation_vectorized(original_model, tokenizer, original_continuation, vocab_size,prompt=None)
   print("Output #{}: ".format(i), generate_with_logits(original_model, tokenizer, "<s>", max_new_tokens=25, noise=noise, first_idx = 0))
   print("==========================================")

In [ ]:
print(tokenizer.decode(13))

In [ ]:
orig_outputs, intervened_outputs = reft_model(
   tokenizer("[INST] <<SYS>>\nYou are a helpful assistant.\n<</SYS>>\nTell me something. [/INST]\nI:", return_tensors="pt").to('cuda'),
   output_original_output=True
)

In [ ]:
intervened_outputs.logits.shape

In [ ]:
generate_with_logits(reft_model, tokenizer, prompt_no_input_template%"Tell me a story about a man.", max_new_tokens=50, noise=None, first_idx = None)